In [1]:
import pandas as pd
import numpy as np

#전처리 완료 파일 불러오기
df = pd.read_excel('../data/A_preprocessing.xlsx')
df.head()

,id,content,visit_dt,visit_tm,visit_cnt,keyword
0,임채채,가성비도 있고 분위기좋고 아늑함!연간 행사하기 좋은 성남 최고 파티룸,2025년 9월 26일 금요일,"(금,토)올나잇 타임 18~08",1번째 방문,"['가성비가 좋아', '인테리어가 멋져', '매장이 넓어', '아늑해', '방음이 ..."
1,kom****,공간이 너무 예쁘고 깔끔해요! 노래방 기계도 최신식이라 노래방이랑 거의 똑같았어요~...,2025년 9월 20일 토요일,"(금,토)올나잇 타임 18~08",1번째 방문,"['인테리어가 멋져', '시설이 깔끔해']"
2,dud****,일요일에 친구들이랑 아이콘 매치를 보려고모란에 파티룸을 찾았는데사장님이 친절하시고 ...,2025년 9월 14일 일요일,(일~목)올나잇 18~08,1번째 방문,"['가성비가 좋아', '인테리어가 멋져', '친절해', '컨셉이 독특해', '대화하..."
3,공백13,아버지 환갑행사로 가족들이 많았는데 저희끼리만 단독으로 있어 분위기도 좋고 행사하기...,2025년 9월 13일 토요일,"(금,토)올나잇 타임 18~08",1번째 방문,"['인테리어가 멋져', '대화하기 좋아', '단체모임 하기 좋아', '시설이 깔끔해..."
4,rha****,세번째 재방문입니당공간도 넓고 인원수 맞춰서 준비해주셔서 너무 좋아용다음에 또 올게...,2025년 9월 8일 월요일,(일~목)올나잇 18~08,1번째 방문,"['인테리어가 멋져', '단체모임 하기 좋아', '오래 머무르기 좋아', '시설이 ..."


-분석단계-<br>
1.파티룸 수요 분석<br>
2.충성도 지표(재방문) 분석<br>
3.키워드 분석<br>
4.이벤트유형 분석<br>
5.긍*부정 요인 분석

In [2]:
#파티룸 수요 분석

#visit_dt에서 월, 요일을 떼어내어 인기있는 월, 요일을 파악
df['month'] = df['visit_dt'].str.split().str[1]
df['date'] = df['visit_dt'].str.split().str[3]

#visit_dt 그룹화
mon_grouped = df.groupby(['month'])['visit_dt'].count().reset_index()
dt_grouped = df.groupby(['date'])['visit_dt'].count().reset_index()
tm_grouped = df.groupby(['visit_tm'])['visit_dt'].count().reset_index()

#그룹화하여 랭킹
hot_mon = mon_grouped.nlargest(5,'visit_dt') #월별 랭킹
hot_date = dt_grouped.nlargest(5,'visit_dt') #요일별 랭킹
tm_grouped #시간대별 

#비율(per) 추가
hot_mon['per'] = round(hot_mon['visit_dt']/len(df)*100,2)
hot_date['per'] = round(hot_date['visit_dt']/len(df)*100,2)
tm_grouped['per'] = round(tm_grouped['visit_dt']/len(df)*100,2)
tm_grouped


,visit_tm,visit_dt,per
0,"(금,토)올나잇 타임 18~08",48,20.87
1,(월~일)데이 타임 10~17,43,18.70
2,(일~목)올나잇 18~08,42,18.26
3,unknown,97,42.17


In [3]:
#충성도 지표(재방문) 분석

#리뷰 내용 중 재방문 관련 언급이 있는 리뷰 추출 / 휴리스틱 방식(직접 정의)
df['retention'] = df['content'].str.contains('또|재방문|늘|의사|다음|다시|단골|여기만')
df_true = df[df['retention']]

#재방문 언급률
#재방문 언급
true_cnt = len(df_true)

#전체
total_cnt = len(df)

#언급 비율
ratio = round(true_cnt / total_cnt *100, 2)

print(f'재방문 언급 리뷰 수 : {true_cnt}')
print(f'전체 리뷰 수 : {total_cnt}')
print(f'재방문 언급 비율 : {ratio}')

재방문 언급 리뷰 수 : 94
전체 리뷰 수 : 230
재방문 언급 비율 : 40.87


In [4]:
#리뷰 작성자별 방문 횟수
retention_grouped = df.groupby(['visit_cnt'])['id'].count().reset_index()
retention_grouped['per'] = round(retention_grouped['id']/len(df)*100, 2)
retention_grouped

,visit_cnt,id,per
0,1번째 방문,206,89.57
1,2번째 방문,18,7.83
2,3번째 방문,5,2.17
3,4번째 방문,1,0.43


In [ ]:
#리뷰 작성자 중 실제 재방문률(A)
re_visit = retention_grouped.loc[1,'id'] + retention_grouped.loc[2,'id'] + retention_grouped.loc[3,'id']
#리뷰 작성자 중 실제 재방문률(B) 
# re_visit = retention_grouped.loc[1,'id']
retention_per = round(re_visit/len(df) * 100, 2)
print(f'실제 재방문률 : {retention_per}(리뷰수 기준)')

실제 재방문률 : 10.43(리뷰수 기준)


In [6]:
import ast
#문자열로 저장된 리스트를 실제 리스트로 변환
print(type(df['keyword'][0]))
df['keyword'] = df['keyword'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
print(type(df['keyword'][0]))

<class 'str'>
<class 'list'>


In [7]:
#키워드 분석
#df['keyword'] 행 안에 있는 리스트 들을 꺼내어 열 단위로 펼치기,'키워드 없음' 제외
df_exploded = df.explode('keyword') 
df_exploded = df_exploded[df_exploded['keyword'] != '키워드 없음']

#키워드 빈도 세기
keyword_fre = df_exploded['keyword'].value_counts().reset_index()
keyword_fre['per'] = round(keyword_fre['count']/keyword_fre['count'].sum()*100,2)
keyword_fre

,keyword,count,per
0,인테리어가 멋져,121,16.20
1,가성비가 좋아,103,13.79
2,매장이 넓어,82,10.98
3,시설이 깔끔해,71,9.50
4,단체모임 하기 좋아,68,9.10
5,조명이 좋아,56,7.50
6,친절해,55,7.36
7,방음이 잘돼,35,4.69
8,룸이 잘 되어있어,24,3.21
9,화장실이 깨끗해,22,2.95


In [8]:
#이벤트 유형 분석
#형태소 분석

from konlpy.tag import Okt
from collections import Counter
okt = Okt()

#df['content'] 행별 데이터 명사 추출
data = []
for row in df['content']:
    r_noun = okt.nouns(row)
    data.append(r_noun)

df['noun'] = data 
df['noun']

0                   [가성, 비도, 분위기, 연간, 행사, 성남, 최고, 파티, 룸]
1                   [공간, 노래방, 기계, 최신, 노래방, 거의, 다음, 또, 거]
2                [일요일, 친구, 아이콘, 매치, 모란, 파티룸, 사장, 룸, 컨디션]
3      [아버지, 환갑, 행사, 가족, 끼리, 단독, 분위기, 행사, 리뷰, 이벤트, 노래...
4                      [세번, 재, 방문, 공간, 인원, 수, 준비, 다음, 또]
                             ...                        
225    [소품, 방, 분위기, 고풍, 데, 다가, 젠, 텔레스트, 레이, 션, 게임, 수,...
226    [마지막, 생일, 여기, 파티룸, 선택, 테이블, 커서, 인원, 수용, 것, 시간,...
227                                                 [내용]
228    [명, 이용, 파티룸, 사진, 커서, 지하, 에어컨, 선풍기, 음식, 사서, 다, ...
229               [밤, 프로젝터, 화면, 커서, 영화, 보기, 에어컨, 빵빵, 가요]
Name: noun, Length: 230, dtype: object

In [9]:
#이벤트 사전 만들기
event_map = {'생일':['생일', '생신', '생파', '생일파티', '환갑'],
             '회식':['회식', '회사', '동료', '친목'],
             '연말/연초모임':['연말모임','연초모임','연말','연초','신년','신년회','송년회'],
             '돌잔치':['돌잔치','백일','첫돌','돌', '잔치'],
             '크리스마스':['크리스마스','성탄','성탄절','xmas','트리', '산타'],
             '프로포즈':['프로포즈','고백','결혼'],
             '브라이덜샤워':['브샤','브라이덜샤워','브라이덜','샤워', '신부'],
             '동창회':['동창회','동창', '대학교', '고등학교'],
             '기타모임':['단체','동아리','소모임','동호회', '청첩장']}

#역매핑
event = {kw:evt for evt, kws in event_map.items() for kw in kws}
event_keys = set(event.keys())

In [10]:
#언급된 이벤트 별로 집계
evt_rv_cnt = Counter()

for tokens in df['noun']:
    mentioned = {event[t] for t in set(tokens) if t in event_keys} #event[t] 키워드를 대표 이벤트로 변환
    evt_rv_cnt.update(mentioned)

evt_rv_cnt

Counter({'연말/연초모임': 27,
         '기타모임': 24,
         '생일': 22,
         '크리스마스': 14,
         '동창회': 4,
         '회식': 4,
         '프로포즈': 1,
         '브라이덜샤워': 1,
         '돌잔치': 1})

In [11]:
#전체 리뷰 중에서 이벤트성 방문 비율
evt_rank = pd.Series(evt_rv_cnt).sort_values(ascending=False)
evt_rank.sum() 
round(evt_rank.sum()/len(df)*100, 2)

np.float64(42.61)

In [12]:
#긍*부정 요인 분석(시설)
facility = ['노래방', '에어컨', '냉장고', '빔프로젝터', '화장실', '인테리어', '보드게임', '쇼파', '조명', '주차', 'PC방', '게임룸', '흡연실'] 

#긍정/부정 감성 단어 사전 
pos_word = ['장점', '갖춘', '추천', '강추', '좋', '좋았', '좋아', '좋더라', '좋습니', '좋고', '굿굿', '최고', '짱', '감탄', '최상', '만족', '깔끔', '깨끗', '깨끗하고', '빵빵', '시원', '쾌적', '이뻐', '이쁘고', '예뻐', '예뻤', '이쁨', '멋진', '힙해요', '매력', '재밌', '즐거웠', '즐겼' '넓고', '커서', '잘', '가성비','저렴', '괜찮', '편했', '편히', '편합', '아늑하고', '아늑한', '사진찍기', '포토존', '인생샷', '핫하', '딱이네', '빠르게', '신속하게', '매끄럽게', '빠르고', '빨라', '가능'] 
neg_word = ['단점', '빼고는', '아쉽', '아쉬웠', '아쉬워', '불만족', '별로', '글쎄', '더러워', '부족', '시끄럽', '시끄러', '비싸', '비싼', '짜증', '아깝', '아까웠', '좁다', '좁고', '협소', '협소한데', '습하고', '눅눅한', '눅눅', '불친절', '느리고', '느려', '답답', '힘들어', '힘들었', '어려웠' '주차불가', '불가', '외부에', '외부로', '바깥에', '밖에']


In [13]:
#.,!,? 기준으로 문장 분리
import re

def split_sentence(text):
    sentences = re.split(r'[.!?]|지만', text)
    return [s.strip() for s in sentences if s.strip()]

df['sentences'] = df['content'].apply(split_sentence)

In [14]:
#문장별 감성 분석 함수 생성

def classify_sentiment(sentence):
    if any(word in sentence for word in pos_word): 
        return '긍정'
    elif any(word in sentence for word in neg_word):
        return '부정'
    else:
        return '중립'

In [15]:
#시설별 감성 분석 결과 집계
facility_sentiment = Counter()

for sent_list in df['sentences']:
    for sent in sent_list:    
        sentiment = classify_sentiment(sent)
        for f in facility:
            if f in sent:
                facility_sentiment[(f, sentiment)] += 1

facility_sentiment

Counter({('노래방', '긍정'): 79,
         ('인테리어', '긍정'): 37,
         ('보드게임', '긍정'): 20,
         ('노래방', '중립'): 16,
         ('빔프로젝터', '긍정'): 11,
         ('조명', '긍정'): 10,
         ('냉장고', '긍정'): 7,
         ('주차', '긍정'): 6,
         ('에어컨', '긍정'): 5,
         ('쇼파', '긍정'): 5,
         ('화장실', '긍정'): 5,
         ('인테리어', '중립'): 4,
         ('조명', '중립'): 3,
         ('화장실', '부정'): 2,
         ('쇼파', '중립'): 2,
         ('화장실', '중립'): 2,
         ('보드게임', '중립'): 2,
         ('냉장고', '중립'): 2,
         ('에어컨', '중립'): 1,
         ('빔프로젝터', '중립'): 1})

In [16]:
#피벗테이블
result = pd.DataFrame([(*k,v) for k,v in facility_sentiment.items()], columns = ['시설', '감정', '리뷰수'])
result_pivot = result.pivot(index='시설', columns='감정', values='리뷰수').fillna(0).astype(int)
result_pivot

감정,긍정,부정,중립
시설,,,
냉장고,7,0,2
노래방,79,0,16
보드게임,20,0,2
빔프로젝터,11,0,1
쇼파,5,0,2
에어컨,5,0,1
인테리어,37,0,4
조명,10,0,3
주차,6,0,0


In [17]:
#긍정 요인 랭킹
result_pivot['긍정'].sort_values(ascending=False)

시설
노래방      79
인테리어     37
보드게임     20
빔프로젝터    11
조명       10
냉장고       7
주차        6
쇼파        5
에어컨       5
화장실       5
Name: 긍정, dtype: int64